# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [4]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import cryptography

# Import API key
from api_keys import geoapify_key

In [5]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,3.81,70,78,14.65,TF,1706633019
1,1,yellowknife,62.4560,-114.3525,-4.77,100,100,6.71,CA,1706633019
2,2,albany,42.6001,-73.9662,0.37,84,90,2.19,US,1706632892
3,3,sinabang,2.4803,96.3801,27.04,81,98,2.14,ID,1706633019
4,4,ust-nera,64.5667,143.2000,-35.31,100,99,0.55,RU,1706633019


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [13]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot_3 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1.31,
    color = "City"
    
   
)

# Display the map
# YOUR CODE HERE
map_plot_3

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [14]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
filtered_cities = city_data_df[(city_data_df['Max Temp'] > 25) & (city_data_df['Humidity'] < 70)]
# Drop any rows with null values
# YOUR CODE HERE
filtered_cities = filtered_cities.dropna()
# Display sample data
# YOUR CODE HERE
filtered_cities.sample(5)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
533,533,riberalta,-10.9833,-66.1000,32.11,66,75,2.06,BO,1706633141
406,406,massakory,12.9960,15.7293,27.72,13,1,5.47,TD,1706633098
88,88,camayenne,9.5350,-13.6878,32.05,51,66,4.12,GN,1706633035
166,166,mazagao,-0.1150,-51.2894,30.03,56,86,2.56,BR,1706633050
579,579,acajutla,13.5928,-89.8275,30.05,43,86,1.23,SV,1706633149


### Step 3: Create a new DataFrame called `hotel_df`.

In [15]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = filtered_cities[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df['Hotel Name'] = ""
# Display sample data
# YOUR CODE HERE
hotel_df.sample(5)

,City,Country,Lat,Lng,Humidity,Hotel Name
444,cidreira,BR,-30.1811,-50.2056,62,
452,robore,BO,-18.3333,-59.7500,49,
88,camayenne,GN,9.5350,-13.6878,51,
342,ati,TD,13.2154,18.3353,12,
487,alo,NG,7.4178,6.7371,14,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [20]:
# Set parameters to search for a hotel
radius = 5000
params = {
    "radius": radius,
    "type": "lodging",  
    "sort": "distance",
    "limit": 1,
    "apiKey": geoapify_key

}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"lat:{lat} lng:{lng}"
    params["bias"] = f"lat:{lat} lng:{lng}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
villa allende - nearest hotel: No hotel found
illapel - nearest hotel: No hotel found
monrovia - nearest hotel: No hotel found
batie - nearest hotel: No hotel found
puerto ayacucho - nearest hotel: No hotel found
fray luis beltran - nearest hotel: No hotel found
kingston - nearest hotel: No hotel found
vredendal - nearest hotel: No hotel found
alice springs - nearest hotel: No hotel found
puerto armuelles - nearest hotel: No hotel found
camayenne - nearest hotel: No hotel found
raja - nearest hotel: No hotel found
west island - nearest hotel: No hotel found
belo jardim - nearest hotel: No hotel found
sidi ifni - nearest hotel: No hotel found
hermanus - nearest hotel: No hotel found
kavaratti - nearest hotel: No hotel found
artigas - nearest hotel: No hotel found
tidjikja - nearest hotel: No hotel found
punta cana - nearest hotel: No hotel found
mazagao - nearest hotel: No hotel found
san luis de la loma - nearest hotel: No hotel found
saipan - nearest hotel: No ho

,City,Country,Lat,Lng,Humidity,Hotel Name
11,villa allende,AR,-31.2946,-64.2954,38,No hotel found
29,illapel,CL,-31.6308,-71.1653,42,No hotel found
37,monrovia,LR,6.3005,-10.7969,69,No hotel found
40,batie,BF,9.8833,-2.9167,10,No hotel found
57,puerto ayacucho,VE,5.6639,-67.6236,35,No hotel found
...,...,...,...,...,...,...
576,saminaka,NG,10.4140,8.6828,18,No hotel found
578,muyuka,CM,4.2898,9.4103,46,No hotel found
579,acajutla,SV,13.5928,-89.8275,43,No hotel found
585,caleta olivia,AR,-46.4393,-67.5281,32,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [26]:
# Configure the map plot
map_plot = hotel_df.hvplot.points(
    x="Lng",
    y="Lat",
    geo=True,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    title="City Data Map with Hotels",
    size=20,
    scale = 1.31,
    color="blue",
    hover_cols=['City', 'Country', 'Hotel Name', 'Humidity']  # Add hotel name and country to hover
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name,Humidity)